# Dealing with Notebook Exceeded Allowed Compute when doing TTA
## Or, dealing with Memory Limit exceeded in TensorFlow

During recent competition Cassava Leaf Diseases Classification I've struggled quite a bit overcoming the "Notebook Exceeded Allowed Compute" error, so I wanted to share what helped me.

The error could be two things - either disk or RAM used up. It was definitely not disk since I wasn't writing anything except for a small CSV file.

As for the memory usage...

Here's my code without TTA that simply iterates through examples, no TTA.
(I was using a script and not a notebook, so only excerpts here)

In [ ]:
def run_predictions(model):
    predictions = []
    test_image_names = []

    test_dir = os.path.join(INPUT_FOLDER, "test_images")

    for image_name in os.listdir(test_dir):
        image = tf.keras.preprocessing.image.load_img(os.path.join(test_dir, image_name), target_size=IMAGE_SIZE)
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = tf.keras.applications.inception_v3.preprocess_input(image)

        cur_prediction = model.predict(image)

        predictions.append(cur_prediction[0])
        test_image_names.append(image_name)

    return predictions, test_image_names

So, to add TTA I did very little, in fact:
- on each iteration, make 2 (or more) copies of the image.
- have `ImageDataGenerator` augment them and then predict.

Below only changed parts:

In [ ]:
def run_predictions(model):
    ...

    datagen = ImageDataGenerator(...)

    for image_name in os.listdir(test_dir):
        ...

        images = np.broadcast_to(image, (N_FOLD,) + image.shape[1:])
        cur_prediction = model.predict(datagen.flow(images))
        cur_prediction = np.mean(cur_prediction, axis=0, keepdims=True)

        ...

    return predictions, test_image_names

As you can see, there's `broadcast_to` the number of folds for TTA, and then predict on them having `datagen` augment them.

This code, even though on each iterations there are only <= 10 copies of the image, started running into the memory error, which was not right.

I ran the experiment on my dev machine (without GPU :( ), and after a few hours I could see almost all of my RAM consumed, so I had a repro.

However, when I started removing parts, I still had the same problem, both locally and on Kaggle, until I removed `model.predict`.

So, I started researching online, and apparently, not only I am facing the same problem:
- [Repeatedly calling model.predict(...) results in memory leak](https://github.com/keras-team/keras/issues/13118)
- [Memory leak on TF 2.0 with model.predict or/and model.fit with keras](https://github.com/tensorflow/tensorflow/issues/33030)
- [Memory leak on TF 2.0 with model.predict or/and model.fit with keras](https://github.com/tensorflow/tensorflow/issues/34579)

And even though some of the issues are closed, and the comments say there were a few fixes, evidently people are still hitting the issues in some cases.

So, the advice from there that worked for me was to use `predict_on_batch` (there's also `gc.collect` but it didn't work on its own):

In [ ]:
def run_predictions(model):
    ...

    datagen = ImageDataGenerator(...)

    for image_idx, image_name in enumerate(os.listdir(test_dir)):
        ...

        images = np.broadcast_to(image, (N_FOLD,) + image.shape[1:])
        # Use predict_on_batch since looks like predict has a memory leak...
        prediction = model.predict_on_batch(next(datagen.flow(images)))
        ...

        if image_idx % 100 == 0:  # Only run every so many images.
            gc.collect()

    return predictions, test_image_names